In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
import fiftyone as fo
import fiftyone.zoo as foz
import os
import logging
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

**Logging setup**

In [5]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
os.environ["FIFTYONE_DEFAULT_DATASET_DIR"] = "Z:/open_images_v7"
fo.config.dataset_zoo_dir = "Z:/open_images_v7"

### Data augmentation and preprocessing

In [10]:
def augment_image(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image


In [15]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = augment_image(image)
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

In [16]:
def prepare_dataset(dataset, batch_size=32, shuffle_buffer=1000):
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

### Load Model

**Load Food101**

In [12]:
food101_dataset = tfds.load('food101', split=['train', 'validation'], as_supervised=True)

INFO:absl:Load dataset info from C:\Users\Troxi\tensorflow_datasets\food101\2.0.0
INFO:absl:Reusing dataset food101 (C:\Users\Troxi\tensorflow_datasets\food101\2.0.0)
INFO:absl:Creating a tf.data.Dataset reading 32 files located in folders: C:\Users\Troxi\tensorflow_datasets\food101\2.0.0.
INFO:absl:Creating a tf.data.Dataset reading 16 files located in folders: C:\Users\Troxi\tensorflow_datasets\food101\2.0.0.
INFO:absl:Constructing tf.data.Dataset food101 for split ['train', 'validation'], from C:\Users\Troxi\tensorflow_datasets\food101\2.0.0


**Load Open Images V7**

In [13]:
food_classes = ["Egg (Food)", "Fast food", "Food", "Seafood",]

In [ ]:
open_v7_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    splits=["train", "validation"],
    classes=food_classes,
    max_samples=10000  # Adjust this number as needed
)

### Combine Datasets

In [ ]:
x_food101, y_food101 = load_and_preprocess_data(food101_dataset['train'])
x_open_v7, y_open_v7 = load_and_preprocess_data(open_v7_dataset['train'])

x_train = np.concatenate([x_food101, x_open_v7])
y_train = np.concatenate([y_food101, y_open_v7])

In [ ]:
# Split into train and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Create tf.data.Dataset
train_dataset = prepare_dataset(tf.data.Dataset.from_tensor_slices((x_train, y_train)))
val_dataset = prepare_dataset(tf.data.Dataset.from_tensor_slices((x_val, y_val)))

### Model architecture

In [ ]:
num_classes = len(np.unique(y_train))

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

**Training**

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1
)

callbacks = [
    checkpoint,
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3)
]

In [ ]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_dataset, epochs=20, validation_data=val_dataset, callbacks=callbacks)


**Inference**

In [ ]:
def predict_food(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    
    return predicted_class

### Nutrition data integration

In [ ]:
nutrition_data = pd.read_csv("Nutrition-Data/nutrients_csvfile.csv")

def get_nutrition_info(food_item):
    try:
        nutrition = nutrition_data[nutrition_data['food_item'] == food_item].iloc[0]
        return {
            'calories': nutrition['calories'],
            'protein': nutrition['protein'],
            'carbs': nutrition['carbs'],
            'fat': nutrition['fat']
        }
    except IndexError:
        return get_nutrition_info_from_api(food_item)

# API CALL WHEN NUTRITION VALUERS ARE NOT IN THE LOCAL DATASET
API_KEY = "hydUyBjWVdUlt1qNIeB2dKGgQYbjFiQwMjm6YpBn" 
API_ENDPOINT = "https://api.nal.usda.gov/fdc/v1/foods/search"

def get_nutrition_info_from_api(food_item):
    params = {
        "api_key": API_KEY,
        "query": food_item,
        "dataType": ["Survey (FNDDS)"],
        "pageSize": 1
    }
    
    response = requests.get(API_ENDPOINT, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['foods']:
            food = data['foods'][0]
            nutrients = food['foodNutrients']
            
            nutrition_info = {
                'calories': next((n['value'] for n in nutrients if n['nutrientName'] == 'Energy'), None),
                'protein': next((n['value'] for n in nutrients if n['nutrientName'] == 'Protein'), None),
                'carbs': next((n['value'] for n in nutrients if n['nutrientName'] == 'Carbohydrate, by difference'), None),
                'fat': next((n['value'] for n in nutrients if n['nutrientName'] == 'Total lipid (fat)'), None)
            }
            
            return nutrition_info
    
    # If API call fails or no data found, return None
    return None

### Main function

In [ ]:
def food_recognition_and_nutrition(image_path):
    try:
        predicted_class = predict_food(image_path)
        food_item = class_labels[predicted_class]  # You need to define class_labels
        nutrition_info = get_nutrition_info(food_item)
        
        return {
            'food_item': food_item,
            'nutrition_info': nutrition_info
        }
    except Exception as e:
        logger.error(f"Error in food recognition: {str(e)}")
        return None

### TFLITE Model

In [ ]:
# TFLite conversion
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('food_recognition_model_v2.tflite', 'wb') as f:
    f.write(tflite_model)

print("TensorFlow Lite model saved successfully.")